In [1]:
import os
import numpy as np
import pandas as pd

In [4]:
src, tgt = [], []

with open('/data/ased/polar_nmt/polar_src_nocontext_1550179785.txt', 'r') as infile:
    for line in infile:
        src.append(line.strip())
        
with open('/data/ased/polar_nmt/polar_tgt_nocontext_1550179785.txt', 'r') as infile:
    for line in infile:
        tgt.append(line.strip())
        
print(len(src))
print(len(tgt))

3112578
3112576


In [8]:
print(src[3112577])
print(tgt[-1])

Oh damn , I'm really sorry to hear that . I understand your situation now

It's okayy



In [10]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=10)
train_idx, valid_idx = next(kfold.split(src[:len(tgt)], tgt))

print(len(train_idx), len(valid_idx))

2801318 311258


In [13]:
train_idx = set(train_idx)
valid_idx = set(valid_idx)

train_file = open('polar_train.txt', encoding='utf8', mode='w')
valid_file = open('polar_valid.txt', encoding='utf8', mode='w')

for line_ix, (src_, tgt_) in enumerate(zip(src, tgt)):
    if line_ix in train_idx:
        train_file.write(src_.strip())
        train_file.write('\t')
        train_file.write(tgt_.strip())
        train_file.write('\n')
    elif line_ix in valid_idx:
        valid_file.write(src_.strip())
        valid_file.write('\t')
        valid_file.write(tgt_.strip())
        valid_file.write('\n')
        
train_file.close()
valid_file.close()

In [12]:
pwd

'/home/kyle.shaffer/keras_transformer_experiment'

In [15]:
from collections import Counter

polar_vocab = Counter()

for s, t in zip(src, tgt):
    polar_vocab.update(s.split())
    polar_vocab.update(t.split())
    
print(len(polar_vocab))
polar_vocab.most_common(20)

423373


[('I', 3763446),
 ('to', 3189683),
 ('you', 2849935),
 ('.', 2729416),
 (',', 2231125),
 ('and', 2140348),
 ('a', 1839914),
 ('?', 1678205),
 ('the', 1556114),
 ('i', 1465279),
 ('it', 1414765),
 ('that', 1370563),
 ('me', 1224388),
 ('is', 1058897),
 ('of', 981471),
 ('my', 974931),
 ('in', 881798),
 ('have', 844683),
 ('for', 839288),
 ('but', 808126)]

In [16]:
with open('/data/users/kyle.shaffer/dialog_data/polar_vocab.txt', encoding='utf8', mode='w') as outfile:
    for w, w_cnt in polar_vocab.most_common():
        outfile.write(w)
        outfile.write('\t')
        outfile.write(str(w_cnt))
        outfile.write('\n')

In [17]:
vocab_df = pd.DataFrame(polar_vocab.most_common(), columns=['word', 'word_count'])
print(vocab_df.shape)
vocab_df.head(10)

(423373, 2)


,word,word_count
0,I,3763446
1,to,3189683
2,you,2849935
3,.,2729416
4,",",2231125
5,and,2140348
6,a,1839914
7,?,1678205
8,the,1556114
9,i,1465279


In [22]:
vocab_df[vocab_df.word_count > 5].shape

(101449, 2)

In [57]:
valid_df = pd.read_csv('/data/users/kyle.shaffer/dialog_data/polar_valid.txt', sep='\t', encoding='utf8', names=['left', 'right'])
print(valid_df.shape)
valid_df.head(20)

(311238, 2)


,left,right
0,And go where ?,Mums Sisters Shelter
1,Mums Sisters Shelter,Mom lives in a group home
2,Mom lives in a group home,She would make room shes family and you're abused
3,She would make room shes family and you're abused,Shelters here only take women who have been be...
4,Shelters here only take women who have been be...,A sofa A floor You are beaten
5,A sofa A floor You are beaten,"Ugggh , do you even know what a studio apartme..."
6,"Ugggh , do you even know what a studio apartme...",Studio here means one bedroom bathroom living ...
7,Studio here means one bedroom bathroom living ...,One bedroom for them would be luxury
8,On may,there is still a lot of time you will prepare ...
9,there is still a lot of time you will prepare ...,I made lots mistkakes in that exercise yesterday


In [58]:
valid_df.tail(10)

,left,right
311228,California,Wow ! You're lucky ! So good morning !
311229,Wow ! You're lucky ! So good morning !,It's morning there ?
311230,It's morning there ?,Yes .
311231,Yes .,Europe ?
311232,Europe ?,"7.15 am yes , Europe"
311233,"7.15 am yes , Europe",France
311234,France,Nope
311235,Nope,Dang Hmm
311236,Dang Hmm,it's ok
311237,it's ok,I mean I think I at least guessed the right ti...


In [53]:
valid_df = valid_df[(valid_df.left.notnull()) & (valid_df.right.notnull()) & (valid_df.left != '') & (valid_df.right != '')]

In [71]:
valid_df[valid_df.left == '\n'].shape

(0, 2)

In [56]:
valid_df.to_csv('/data/users/kyle.shaffer/dialog_data/polar_valid.txt', sep='\t', index=False, header=False)

In [139]:
valid_df['left'] = valid_df.left.apply(lambda x: x.replace('\n', '').replace('\t', ''))
valid_df['right'] = valid_df.right.apply(lambda x: x.replace('\n', '').replace('\t', ''))
valid_df.head(10)

,left,right
0,And go where ?,Mums Sisters Shelter
1,Mums Sisters Shelter,Mom lives in a group home
2,Mom lives in a group home,She would make room shes family and you're abused
3,She would make room shes family and you're abused,Shelters here only take women who have been be...
4,Shelters here only take women who have been be...,A sofa A floor You are beaten
5,A sofa A floor You are beaten,"Ugggh , do you even know what a studio apartme..."
6,"Ugggh , do you even know what a studio apartme...",Studio here means one bedroom bathroom living ...
7,Studio here means one bedroom bathroom living ...,One bedroom for them would be luxury
8,On may,there is still a lot of time you will prepare ...
9,there is still a lot of time you will prepare ...,I made lots mistkakes in that exercise yesterday


In [142]:
train_df.shape

(2800385, 2)

In [140]:
valid_df.to_csv('/data/users/kyle.shaffer/dialog_data/polar_valid.txt', sep='\t', index=False, header=False)

In [131]:
train_df = pd.read_csv('/data/users/kyle.shaffer/dialog_data/polar_train.txt', sep='|', names=['left', 'right'])
print(train_df.shape)
train_df.head(10)

(2801100, 2)


,left,right
0,I mean I think I at least guessed the right ti...,that's true
1,that's true,So what's up ?
2,So what's up ?,"well , nothing really Just wanted to say good ..."
3,"well , nothing really Just wanted to say good ...","Oh , good morning :)"
4,"Oh , good morning :)",And have a LOVELY day and the weekend !
5,And have a LOVELY day and the weekend !,Haha thanks !
6,Haha thanks !,:-D That's all Can't go outside so I'm a bit b...
7,:-D That's all Can't go outside so I'm a bit b...,What ? Not famous enough for you ? :P
8,What ? Not famous enough for you ? :P,"Yes , sure I admire them it would be so cool t..."
9,"Yes , sure I admire them it would be so cool t...",So definitely not France


In [73]:
train_df[train_df.left== '\n'].shape

(0, 2)

In [31]:
train_df.tail(10)

,left,right
2800559,What is base 3 and 4 ?,So tell me .
2800560,Kissing and other things I'm sorry if you're f...,About what ? We've been together for 6 months ...
2800561,It's ok,Did you talk to him about this .. ?
2800562,So tell me .,Yeaa but he's still angry I guess And sometime...
2800563,About what ? We've been together for 6 months ...,Of course he'll feel that way . It's been six ...
2800564,Did you talk to him about this .. ?,Well sometimes he makes some infidelity jokes ...
2800565,Yeaa but he's still angry I guess And sometime...,What part are your parents playing in this .. ?
2800566,Of course he'll feel that way . It's been six ...,My dad he jus went off with another woman
2800567,Well sometimes he makes some infidelity jokes ...,"Oh damn , I'm really sorry to hear that . I un..."
2800568,What part are your parents playing in this .. ?,It's okayy


In [133]:
train_df = train_df[(train_df.left.notnull()) & (train_df.right.notnull()) & (train_df.left != '') & (train_df.right != '')]
print(train_df.shape)

(2800395, 2)


In [134]:
train_df = train_df[(train_df.left != ' ') & (train_df.right != ' ')]
print(train_df.shape)

(2800385, 2)


In [74]:
train_df = train_df[(train_df.left != '\n') & (train_df.right != '\n')]
print(train_df.shape)

(2800535, 2)


In [37]:
train_df[(train_df.left.notnull()) & (train_df.right.notnull()) & (train_df.left != '') & (train_df.right != '')].to_csv('/data/users/kyle.shaffer/dialog_data/polar_train.txt', sep='\t', index=False, header=False)

In [135]:
train_path = '/data/users/kyle.shaffer/dialog_data/polar_train.txt'
train_df.to_csv(train_path, sep='\t', index=False, header=False)

In [127]:
train_df['left'] = train_df.left.apply(lambda x: x.replace('\n', '').replace('\t', ''))
train_df['right'] = train_df.left.apply(lambda x: x.replace('\n', '').replace('\t', ''))

In [110]:
with open(train_path, encoding='utf8', mode='w') as outfile:
    for l, r in zip(train_df.left.tolist(), train_df.right.tolist()):
        outfile.write(l.strip())
        outfile.write('|')
        outfile.write(r.strip())
        outfile.write('\n')

In [111]:
train_lines = []
with open(train_path, encoding='utf8', mode='r') as infile:
    for line in infile:
        train_lines.append(line)

In [112]:
len(train_lines)

2801100

In [117]:
train_lines[95440]

" Lmao i don't even think I've seen his face yet , he's over six foot that means instant sex l\n"

In [81]:
s = "Lmao i don't even think I've seen his face yet , he's over six foot that means instant sex l"

train_df[train_df.left == s]

,left,right


In [132]:
train_df.loc[95440]

left      Lmao i don't even think I've seen his face ye...
right                                                  NaN
Name: 95440, dtype: object

In [ ]:
train_df